In [ ]:
#!/usr/bin/env python
import ROOT, sys, os
from ROOT import std

from larlite import larlite
from larcv import larcv
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.path as path
colormap=['blue','red','magenta','green','orange','yellow','pink']
nc=len(colormap)

In [ ]:
iom=larcv.IOManager(larcv.IOManager.kBOTH)        
iom.reset()
iom.set_verbosity(0)                                                                          
iom.add_in_file("/Users/vgenty/Desktop/mcc7_bnb_detsim_to_larcv_hires_v00_p00_out_0000.root")
iom.set_out_file("/tmp/trash.root")
iom.initialize()
iom.read_entry(10)

larbysimg=larcv.LArbysImage()
cfg=larcv.CreatePSetFromFile("unit.fcl","LArbysImage")
larbysimg.configure(cfg)
larbysimg.initialize()

In [ ]:
fig,ax=plt.subplots(figsize=(20,20))

img=iom.get_data(larcv.kProductImage2D,"tpc")

im=larcv.as_ndarray(img.Image2DArray()[0])

plt.imshow(np.where(im>10.0,1.0,0.0).astype(np.uint8),cmap='Greys',interpolation='None')

nz_pixels=np.where(im>0.0)
ax.set_ylim(np.min(nz_pixels[0])-10,np.max(nz_pixels[0])+10)
ax.set_xlim(np.min(nz_pixels[1])-10,np.max(nz_pixels[1])+10)
plt.show()

In [ ]:
larbysimg.process(iom)

In [ ]:
fig,ax=plt.subplots(figsize=(20,20))

#Get the image
shape_img=np.where(im>10.0,1.0,0.0).astype(np.uint8)
shape_img[:,::-1]=shape_img
plt.imshow(shape_img,cmap='Greys',interpolation='none')
nz_pixels=np.where(shape_img>0.0)
ax.set_ylim(np.min(nz_pixels[0])-10,np.max(nz_pixels[0])+10)
ax.set_xlim(np.min(nz_pixels[1])-10,np.max(nz_pixels[1])+10)

mgr=larbysimg.Manager()
dm=mgr.DataManager()
data=dm.Data(0)

all_p0=data._all_ctors_v_v[0]

for ctor in all_p0:
    pts=np.array([[pt.x,pt.y] for pt in ctor])
    plt.plot(pts[:,0],pts[:,1],'-o')

ax.set_aspect(2.0)
ax.set_title("ALL Contours")
plt.show()

In [ ]:
fig,ax=plt.subplots(figsize=(20,20))

#Get the image
shape_img=np.where(im>10.0,1.0,0.0).astype(np.uint8)
shape_img[:,::-1]=shape_img
plt.imshow(shape_img,cmap='Greys',interpolation='none')
nz_pixels=np.where(shape_img>0.0)
ax.set_ylim(np.min(nz_pixels[0])-10,np.max(nz_pixels[0])+10)
ax.set_xlim(np.min(nz_pixels[1])-10,np.max(nz_pixels[1])+10)

mgr=larbysimg.Manager()
dm=mgr.DataManager()
data=dm.Data(1)

split_lines=data._split_line_v_v[0]

for sline in split_lines:
    ax.plot([0,900],[sline.y(0),sline.y(900)])

atomic_p0=data._atomic_ctor_v_v[0]

for atomic in atomic_p0:
    pts=np.array([[pt.x,pt.y] for pt in atomic])
    plt.plot(pts[:,0],pts[:,1],'-o')

ax.set_aspect(2.0)
ax.set_title("ALL Contours")
plt.show()

In [ ]:
fig,ax=plt.subplots(figsize=(20,20))
shape_img=np.where(im>50.0,1.0,0.0).astype(np.uint8)
shape_img[:,::-1]=shape_img
plt.imshow(shape_img,cmap='Greys',interpolation='none')
nz_pixels=np.where(shape_img>0.0)
ax.set_ylim(np.min(nz_pixels[0])-10,np.max(nz_pixels[0])+10)
ax.set_xlim(np.min(nz_pixels[1])-10,np.max(nz_pixels[1])+10)

cv=larbysimg.Manager().GetClusterAlg(2)
mgr=larbysimg.Manager()
dm=mgr.DataManager()
cv_data=dm.Data(2)

cv_ctor_lines_v=cv_data._ctor_lines_v_v_v[0]
cv_inter_pts_v=cv_data._ipoints_v_v_v[0]
ix=-1

for cv_ctor_lines in cv_ctor_lines_v:
    
    ix+=1
    color=colormap[ix]
    
    for line in cv_ctor_lines:
        plt.plot([0,900],[line.y(0),line.y(900)],alpha=0.7,color=color)

    for inter_pt in cv_inter_pts_v[ix]:
        plt.plot(inter_pt.x,inter_pt.y,'o',markersize=10,color=color,alpha=0.9)   
        
ax=plt.gca()
plt.show()

In [ ]:
import matplotlib.patches as patches
fig,ax=plt.subplots(figsize=(20,20))
shape_img=np.where(im>10.0,1.0,0.0).astype(np.uint8)
shape_img[:,::-1]=shape_img
plt.imshow(shape_img,cmap='Greys',interpolation='none')
nz_pixels=np.where(shape_img>0.0)
ax.set_ylim(np.min(nz_pixels[0])-10,np.max(nz_pixels[0])+10)
ax.set_xlim(np.min(nz_pixels[1])-10,np.max(nz_pixels[1])+10)

ix=-1

for cv_ctor_lines in cv_ctor_lines_v:
    
    ix+=1
    color=colormap[ix]
    for line in cv_ctor_lines:
        plt.plot([0,900],[line.y(0),line.y(900)],alpha=0.7,color=color)
        
    for inter_pt in cv_inter_pts_v[ix]:
        plt.plot(inter_pt.x,inter_pt.y,'o',markersize=10,color=color,alpha=0.9)   
        

cv=larbysimg.Manager().GetClusterAlg(3)
mgr=larbysimg.Manager()
dm=mgr.DataManager()
cv_data=dm.Data(3)

circledata_v=cv_data._circledata_v_v[0]
       
for circle in circledata_v:
    c=patches.Circle((circle.center.x,circle.center.y),
                  circle.radius,ec='cyan',fc='none',lw=10)
    ax.add_patch(c)

ax=plt.gca()
ax.set_aspect(2.0)
plt.show()